<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Prétraitement des données</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">

# <b>1. Configuration du notebook

In [1]:
# Imports des modules du projet
import config, src

# Imports classiques
import numpy as np
import pandas as pd

# Imports pour la visualisation
import plotly.io as pio
import plotly.express as px

# Imports spécifiques
import holidays

In [2]:
# Configuration d'un template par défaut pour la visualisation 
pio.templates['custom'] = pio.templates['plotly_white']
pio.templates['custom'].layout.colorway = px.colors.qualitative.Prism

pio.templates.default = 'custom'

---
# <b>2. Chargement et vérifications</b>

In [3]:
# Chargement des données brutes
data = src.load_data(config.RAW_DATA_FILE)

Données chargées avec succès.


In [4]:
# Vérification de valeurs manquantes dans les données
src.check_missing_values(data, print_per_column=False)

Nombre total de valeurs manquantes : 0


In [5]:
# Vérification de doublons dans les données
data.duplicated().value_counts()

False    19735
Name: count, dtype: int64

---
# <b>3. Renommage des variables</b>

Nous avons choisi de renommer les variables du jeu de données afin de les rendre plus compréhensibles et explicites, facilitant ainsi leur manipulation.

Au préalable, nous supprimons les variables `rv1` et `rv2`, qui sont des variables aléatoires initialement introduites dans le jeu de données pour effectuer des tests avec l'algorithme Boruta de sélection de variables. 

In [6]:
# Suppression des variables aléatoires
data.drop(columns=['rv1', 'rv2'], inplace=True)

In [7]:
rename_dict = {
    'date': 'Date',
    'Appliances': 'Energy',
    'lights': 'Lights',
    'T1': 'Kitchen_T',
    'RH_1': 'Kitchen_H',
    'T2': 'Living_T',
    'RH_2': 'Living_H',
    'T3': 'Laundry_T',
    'RH_3': 'Laundry_H',
    'T4': 'Office_T',
    'RH_4': 'Office_H',
    'T5': 'Bath_T',
    'RH_5': 'Bath_H',
    'T6': 'Outside_T',
    'RH_6': 'Outside_H',
    'T7': 'Ironing_T',
    'RH_7': 'Ironing_H',
    'T8': 'TeenRoom_T',
    'RH_8': 'TeenRoom_H',
    'T9': 'ParentsRoom_T',
    'RH_9': 'ParentsRoom_H',
    'T_out': 'Weather_T',
    'Press_mm_hg': 'Weather_P',
    'RH_out': 'Weather_H',
    'Windspeed': 'Weather_WindSpeed',
    'Visibility': 'Weather_Visibility',
    'Tdewpoint': 'Weather_Dewpoint',
}

# Renommage des variables
data.rename(columns=rename_dict, inplace=True)

In [8]:
# Sauvegarde du jeu de données renommé
src.save_data(data, 'renamed_data')

Données sauvegardées avec succès.


---
# <b>4. Feature engineering</b>

In [9]:
# Conversion des dates au format datetime
data['Date'] = pd.to_datetime(data['Date'])

### <b>4.1. Jours de la semaine</b>

In [10]:
day_labels = {
    0: 'Lundi', 
    1: 'Mardi', 
    2: 'Mercredi', 
    3: 'Jeudi', 
    4: 'Vendredi', 
    5: 'Samedi', 
    6: 'Dimanche'
}

# Comptage des occurrences de chaque jour
weekday_counts = data['Date'].dt.weekday.map(day_labels).value_counts().reset_index()
weekday_counts.columns = ['weekday', 'count']

# Graphique en camembert de l'effectif des jours dans le jeu de données
fig = px.pie(
    weekday_counts,
    names='weekday',
    values='count',
    title='Répartition des jours de la semaine',
    category_orders={'weekday': list(day_labels.values())},
    color='weekday'
)

# Mise en page
fig.update_layout(
    showlegend=False
)

# Ajustement des traces
fig.update_traces(
    textinfo='label+percent'
)

# Affichage
fig.show()

In [11]:
# Noms des variables pour l'encodage one-hot
day_var= {
    0: 'monday',
    1: 'tuesday',
    2: 'wednesday',
    3: 'thursday',
    4: 'friday',
    5: 'saturday',
    6: 'sunday'
}

# Variable catégorielle indiquant le jour de la semaine
data['weekday'] = data['Date'].dt.weekday
data['weekday'] = data['weekday'].map(day_var)

# Encodage one-hot de la variable indiquant le jour de la semaine
day_one_hot = pd.get_dummies(data['weekday'], columns=['weekday'], prefix='', prefix_sep='', dtype=int)
day_one_hot = day_one_hot[list(day_var.values())] # Pour avoir les jours dans l'ordre dans le dataframe

# Ajout des variables dans le dataframe
data = pd.concat([data, day_one_hot], axis=1)

# Suppression de la variable utilisée pour la construction
data.drop(columns=['weekday'], inplace=True)

### <b>4.2. Heures</b>

In [12]:
# Variable catégorielle indiquant l'heure
data['hour'] = data['Date'].dt.hour

# Encodage one-hot de la variable indiquant l'heure
hour_one_hot = pd.get_dummies(data['hour'], columns=['hour'], prefix='hour', prefix_sep='_', dtype=int)

# Ajout des variables avec encodage one-hot dans le dataframe
data = pd.concat([data, hour_one_hot], axis=1)

# Transformation sinusoïdale de l'heure
data['sin_hour'] = np.sin(2 * np.pi * data['hour'] / 24)

# Suppression de la variable utilisée pour la construction
data = data.drop(columns=['hour'])

### <b>4.3. Jours fériés et vacances scolaires</b>

In [13]:
# Jours fériés belges en 2016
be_holidays = holidays.Belgium(years=2016)

# Conversion dse dates au format Timestamp 
be_holidays = {pd.Timestamp(date): name for date, name in be_holidays.items()}

# Variable catégorielle indiquant si une entrée est dans un jour férié ou non
data['is_holiday'] = data['Date'].isin(be_holidays).astype(int)

# Nombre de jours fériés dans le jeu de données
n_holidays = data[data['is_holiday'] == 1]['Date'].count()
print(f'Il a {n_holidays} jour(s) férié(s) dans le jeu de données.')

Il a 6 jour(s) férié(s) dans le jeu de données.


In [14]:
# Vacances scolaires belges en 2016
be_school_holidays_2016 = [
    ('2016-02-08', '2016-02-14'),  # Carnaval
    ('2016-03-28', '2016-04-10'),  # Pâques
    ('2016-07-01', '2016-08-31'),  # Été
    ('2016-10-31', '2016-11-06'),  # Toussaint
    ('2016-12-26', '2017-01-08')   # Noël
]

def is_school_holiday(date, holidays):
    """
    Vérifie si une date donnée se situe dans une période de vacances scolaires.

    Args:
        date (pd.Timestamp ou datetime-like): La date à vérifier.
        holidays (list[tuple]): 
            Une liste de tuples représentant les périodes de vacances, où chaque tuple contient une date de 
            début et de fin sous forme de chaîne de caractères compatible avec `pd.Timestamp` (ex. 'YYYY-MM-DD').

    Returns:
        bool: True si la date est incluse dans une période de vacances, sinon False.
    """

    for start, end in holidays:
        if pd.Timestamp(start) <= date <= pd.Timestamp(end):
            return True
    return False

# Variable catégorielle indiquant si une entrée est dans une période de vacances scolaires ou non
data['is_school_holiday'] = data['Date'].apply(lambda x: is_school_holiday(x, be_school_holidays_2016)).astype(int)

# Nombre de jours de vacances scolaires dans le jeu de données
n_school_holidays = len(data.loc[data['is_school_holiday'] == 1, 'Date'].dt.date.unique())
print(f'Il y a {n_school_holidays} jour(s) de vacances scolaires dans le jeu de données.')

Il y a 21 jour(s) de vacances scolaires dans le jeu de données.


### <b>4.4. Saisons</b>

In [15]:
# Périodes des saisons
seasons = {
    0: (pd.Timestamp(year=2016, month=3, day=21), pd.Timestamp(year=2016, month=6, day=20)), # Printemps
    1: (pd.Timestamp(year=2016, month=6, day=21), pd.Timestamp(year=2016, month=9, day=20)), # Été
    2: (pd.Timestamp(year=2016, month=9, day=21), pd.Timestamp(year=2016, month=12, day=20)), # Automne
    3: (pd.Timestamp(year=2016, month=12, day=21), pd.Timestamp(year=2017, month=3, day=20)) # Hiver
}

def get_season(date):
    """
    Détermine la saison correspondant à une date donnée.

    Args:
        date (pd.Timestamp ou datetime-like): La date pour laquelle déterminer la saison.

    Returns:
        int: Le code de la saison associée à la date :
            - 0 : Printemps
            - 1 : Été
            - 2 : Automne
            - 3 : Hiver
    """

    for season, (start, end) in seasons.items():
        if start <= date <= end:
            return season

    return 3 # Hiver pour la période du 1er janvier au 20 mars

# Variable catégorielle indiquant la saison
data['season'] = data['Date'].apply(get_season)

In [16]:
# Nombre de saisons présentes dans le jeu de données
n_seasons = len(data['season'].unique())
print(f'Il y a {n_seasons} saisons différentes dans le jeu de données :')

index_to_season = {
    0 : 'Printemps',
    1: 'Été',
    2: 'Automne',
    3: 'Hiver'
}

# Saisons du jeu de données avec label
seasons = data['season'].map(index_to_season)

# Saisons présentes dans le jeu de données
for season in seasons.unique():
    print(f'- {season}')

Il y a 2 saisons différentes dans le jeu de données :
- Hiver
- Printemps


In [17]:
# Répartition des saisons
season_counts = data['season'].map(index_to_season).value_counts().reset_index()

# Graphique en camembert de l'effectif des jours dans le jeu de données
fig = px.pie(
    season_counts,
    names='season',
    values='count',
    title='Répartition des saisons',
    # category_orders={'weekday': list(config.DAYS.values())},
    color='season'
)

# Mise en page
fig.update_layout(
    showlegend=False
)

# Ajustement des traces
fig.update_traces(
    textinfo='label+percent'
)

# Affichage
fig.show()

In [18]:
# Réencodage des saisons car seulement 2 sont présentes
encode_winter_spring = {
    0: 0,
    3: 1
}
data['season'] = data['season'].replace(encode_winter_spring)

---
# <b>4. Résumé du prétraitement</b>

In [19]:
# Dimension des données prétraitées
n_entries, n_var = data.shape
print(f'Les données prétraitées contiennent {n_entries} entrées pour {n_var} variables.')

Les données prétraitées contiennent 19735 entrées pour 62 variables.


In [20]:
# Sauvegarde du jeu de données prétraitées
src.save_data(data, 'processed_data')

Données sauvegardées avec succès.


---